Want a way to save/load our data.
Can use HDF5 for maximum portability or JLD if we are comfortable staying in the Julia ecosystem.
Some patterns we could use:

1. Have every worker send their data to the driver, and have it write one file
2. Have each worker write its own data file, then collate them at the end using some script
3. Have a group of workers dedicated to collecting data and collating

3 is intermediate between 1 and 2. Each one has advantages and disadvantages. 3 will let us show off some cool Julia features like `WorkerPool` and it's also the most difficult to get right, so let's do that!

In [ ]:
addprocs(6)

IOPool = WorkerPool([2; 3]) # worker "1" is the driver worker
ComputePool = CachingPool([4; 5; 6; 7])

We make `ComputePool` a [`CachingPool`](https://docs.julialang.org/en/latest/stdlib/parallel/#Base.Distributed.CachingPool) so that it can store information about the Hamiltonian between timesteps. Hopefully this will cut down on our runtimes! Now we want to:

1) Have `ComputePool` work on our time evolution function from last time (we'll modify our call to `pmap` to do this).

2) For each eigenvector we get back from `ComputePool`, feed it to `IOPool` to write to disk

3) Have the driver node make a plot for us in the meantime (we already have code to do this!)

It's a good idea to keep track of what version of the code we used to generate the data. 